# Прогнозирование цен потребительского ритейла по тестовой выборке на основе глубоких нейронных сетей
__Выполнил:__ *Домченко Максим*

__Студент группы:__ *РИМ-130962*

In [1]:
# Подключение Google Drive к Colab
from pathlib import Path
from google.colab import drive

drive.mount('/content/drive')
ROOT = Path('/content/drive/MyDrive')

# Создание структуры папок проекта
PROJECT_DIR = ROOT / 'price_forecasting'
DATA = PROJECT_DIR / 'data'
RAW = DATA / 'raw'
PROCESSED = DATA / 'processed'

for directory in [DATA, RAW, PROCESSED]:
    directory.mkdir(parents=True, exist_ok=True)

print('Проектная папка:', PROJECT_DIR)

Mounted at /content/drive
Проектная папка: /content/drive/MyDrive/price_forecasting


In [2]:
# Скачивание датасета
# Цель: загрузить данные из Kaggle

# Добавьте kaggle.json

from google.colab import files
uploaded = files.upload('/root/.kaggle')
!chmod 600 /root/.kaggle/kaggle.json

!kaggle competitions download -p /content/drive/MyDrive/price_forecasting/data/raw -c m5-forecasting-accuracy --force

Saving kaggle.json to /root/.kaggle/kaggle.json
 61% 28.0M/45.8M [00:00<00:00, 289MB/s]
100% 45.8M/45.8M [00:00<00:00, 294MB/s]


In [3]:
# Распаковка архива
# Цель: разархивировать m5-forecasting-accuracy.zip

import zipfile, os

zip_path = "/content/drive/MyDrive/price_forecasting/data/raw/m5-forecasting-accuracy.zip"
extract_dir = "/content/drive/MyDrive/price_forecasting/data/raw"

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Архив распакован в:", extract_dir)

Архив распакован в: /content/drive/MyDrive/price_forecasting/data/raw


In [4]:
# Импорт библиотек для обработки и анализа данных
import numpy as np
import pandas as pd

# Библиотеки для визуализации данных
import matplotlib.pyplot as plt
import seaborn as sns

# Настройки отображения pandas и matplotlib
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)
plt.style.use('ggplot')

# Проверка версий библиотек
print("Версия Pandas:", pd.__version__)
print("Версия NumPy:", np.__version__)

Версия Pandas: 2.2.2
Версия NumPy: 2.0.2


In [5]:
# Пути к файлам исходных данных (исключаем sample_submission)
calendar_path = RAW / 'calendar.csv'
sell_prices_path = RAW / 'sell_prices.csv'
sales_train_validation_path = RAW / 'sales_train_validation.csv'
sales_train_evaluation_path = RAW / 'sales_train_evaluation.csv'

# Загрузка данных в датафреймы pandas
calendar_df = pd.read_csv(calendar_path)
sell_prices_df = pd.read_csv(sell_prices_path)
sales_train_validation_df = pd.read_csv(sales_train_validation_path)
sales_train_evaluation_df = pd.read_csv(sales_train_evaluation_path)

# Проверка успешной загрузки
print('Файлы успешно загружены.')
print('calendar:', calendar_df.shape)
print('sell_prices:', sell_prices_df.shape)
print('sales_train_validation:', sales_train_validation_df.shape)
print('sales_train_evaluation:', sales_train_evaluation_df.shape)

Файлы успешно загружены.
calendar: (1969, 14)
sell_prices: (6841121, 4)
sales_train_validation: (30490, 1919)
sales_train_evaluation: (30490, 1947)


In [6]:
# Просмотр первых строк и общей информации о данных
print("Calendar data:")
display(calendar_df.head())
calendar_df.info()

Calendar data:


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          1969 non-null   object
 1   wm_yr_wk      1969 non-null   int64 
 2   weekday       1969 non-null   object
 3   wday          1969 non-null   int64 
 4   month         1969 non-null   int64 
 5   year          1969 non-null   int64 
 6   d             1969 non-null   object
 7   event_name_1  162 non-null    object
 8   event_type_1  162 non-null    object
 9   event_name_2  5 non-null      object
 10  event_type_2  5 non-null      object
 11  snap_CA       1969 non-null   int64 
 12  snap_TX       1969 non-null   int64 
 13  snap_WI       1969 non-null   int64 
dtypes: int64(7), object(7)
memory usage: 215.5+ KB


In [7]:
print("Sell Prices data:")
display(sell_prices_df.head())
sell_prices_df.info()

Sell Prices data:


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   store_id    object 
 1   item_id     object 
 2   wm_yr_wk    int64  
 3   sell_price  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 208.8+ MB


In [8]:
print("Sales Train Validation data:")
display(sales_train_validation_df.head())
sales_train_validation_df.info()

Sales Train Validation data:


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,...,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,3,0,0,0,1,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,2,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,1,0,...,5,3,1,0,0,0,1,2,3,0,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,2,1,1,0,3,1,1,2,1,1,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: int64(1913), object(6)
memory usage: 446.4+ MB


In [9]:
print("Sales Train Evaluation data:")
display(sales_train_evaluation_df.head())
sales_train_evaluation_df.info()

Sales Train Evaluation data:


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,...,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1,0,0,0,2,0,3,5,0,0,1,1,0,2,1,2,2,1,0,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1,0,0,1,1,0,2,1,0,0,0,0,2,1,3,0,0,1,0,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,1,0,...,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2,0,0,1,2,4,1,6,4,0,0,0,2,2,4,2,1,1,1,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4,1,0,2,3,1,0,3,2,3,1,1,3,2,3,2,2,2,2,0,0,0,2,1,0,0,2,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1947 entries, id to d_1941
dtypes: int64(1941), object(6)
memory usage: 452.9+ MB


## Загрузка и первичный обзор данных выводы:

### 1. Датасет `calendar`:
- **Размер:** `1969` строк и `14` столбцов.
- Присутствуют пропущенные значения в колонках с событиями:  
`event_name_1`, `event_type_1`, `event_name_2`, `event_type_2`.
- Типы данных корректны, но потребуется обработка пропусков.

### 2. Датасет `sell_prices`:
- **Размер:** `6,841,121` строк и `4` столбца.
- Пропущенных значений **нет**.
- Содержит идентификаторы магазинов, товаров, недель года и цену `sell_price`. Типы данных корректны.

### 3. Датасеты `sales_train_validation` и `sales_train_evaluation`:
- **Размер:** по `30,490` строк, столбцов:
  - `validation`: `1919`
  - `evaluation`: `1947`
- Содержат данные о продажах по дням в столбцах формата `d_1, d_2, ..., d_n`.
- Пропущенных значений **нет**, данные корректны и готовы для дальнейшей обработки.

**Итог:** исходные данные загружены корректно и полностью готовы к следующему этапу — базовой очистке и генерации признаков.


In [10]:
# Базовая очистка и преобразование данных в calendar
calendar_df['date'] = pd.to_datetime(calendar_df['date'])
calendar_df['event_name_1'] = calendar_df['event_name_1'].fillna('NoEvent')
calendar_df['event_type_1'] = calendar_df['event_type_1'].fillna('NoEvent')
calendar_df['event_name_2'] = calendar_df['event_name_2'].fillna('NoEvent')
calendar_df['event_type_2'] = calendar_df['event_type_2'].fillna('NoEvent')

# Проверка после преобразования
calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          1969 non-null   datetime64[ns]
 1   wm_yr_wk      1969 non-null   int64         
 2   weekday       1969 non-null   object        
 3   wday          1969 non-null   int64         
 4   month         1969 non-null   int64         
 5   year          1969 non-null   int64         
 6   d             1969 non-null   object        
 7   event_name_1  1969 non-null   object        
 8   event_type_1  1969 non-null   object        
 9   event_name_2  1969 non-null   object        
 10  event_type_2  1969 non-null   object        
 11  snap_CA       1969 non-null   int64         
 12  snap_TX       1969 non-null   int64         
 13  snap_WI       1969 non-null   int64         
dtypes: datetime64[ns](1), int64(7), object(6)
memory usage: 215.5+ KB


## Базовая очистка и преобразование данных в датасете calendar выводы:

- Столбец `date` успешно преобразован в формат datetime.
- Пропущенные значения в столбцах с событиями (`event_name_1`, `event_type_1`, `event_name_2`, `event_type_2`) заменены на строковое значение `'NoEvent'`.
- Все столбцы теперь не содержат пропусков и имеют подходящие типы данных для последующего анализа и моделирования.

Датасет `calendar` полностью подготовлен для дальнейших этапов работы.


In [11]:
# Оптимизация типов данных для sell_prices
sell_prices_df['store_id'] = sell_prices_df['store_id'].astype('category')
sell_prices_df['item_id'] = sell_prices_df['item_id'].astype('category')

# Оптимизация типов данных для sales_train_validation
cat_columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
for col in cat_columns:
    sales_train_validation_df[col] = sales_train_validation_df[col].astype('category')

# Оптимизация типов данных для sales_train_evaluation
for col in cat_columns:
    sales_train_evaluation_df[col] = sales_train_evaluation_df[col].astype('category')

# Проверка оптимизации памяти
print("Sell Prices после оптимизации:")
sell_prices_df.info()

print("\nSales Train Validation после оптимизации:")
sales_train_validation_df.info()

print("\nSales Train Evaluation после оптимизации:")
sales_train_evaluation_df.info()

Sell Prices после оптимизации:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype   
---  ------      -----   
 0   store_id    category
 1   item_id     category
 2   wm_yr_wk    int64   
 3   sell_price  float64 
dtypes: category(2), float64(1), int64(1)
memory usage: 124.0 MB

Sales Train Validation после оптимизации:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: category(6), int64(1913)
memory usage: 446.6 MB

Sales Train Evaluation после оптимизации:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1947 entries, id to d_1941
dtypes: category(6), int64(1941)
memory usage: 453.1 MB


## Оптимизация типов данных выводы:

Выполнена оптимизация типов данных для датасетов:

### 1. Датасет `sell_prices`:
- Столбцы `store_id` и `item_id` преобразованы в категориальный тип (`category`).
- Использование памяти сократилось примерно до `124 MB`.

### 2. Датасет `sales_train_validation`:
- Столбцы `id`, `item_id`, `dept_id`, `cat_id`, `store_id`, `state_id` преобразованы в категориальный тип.
- Использование памяти снизилось примерно до `446.8 MB`.

### 3. Датасет `sales_train_evaluation`:
- Аналогично предыдущему датасету проведена оптимизация столбцов в категориальный тип.
- Использование памяти снизилось примерно до `453.1 MB`.

**Итог:** Оптимизация типов данных позволила существенно сократить использование оперативной памяти, что ускорит последующую обработку и моделирование.

Все датасеты теперь полностью готовы для сохранения и дальнейшего анализа.


In [12]:
# Сохранение обработанных данных в формате parquet
calendar_df.to_parquet(PROCESSED / 'calendar_processed.parquet', index=False)
sell_prices_df.to_parquet(PROCESSED / 'sell_prices_processed.parquet', index=False)
sales_train_validation_df.to_parquet(PROCESSED / 'sales_train_validation_processed.parquet', index=False)
sales_train_evaluation_df.to_parquet(PROCESSED / 'sales_train_evaluation_processed.parquet', index=False)

print("Файлы успешно сохранены в папку:", PROCESSED)

Файлы успешно сохранены в папку: /content/drive/MyDrive/price_forecasting/data/processed


## Итоговые выводы по ноутбуку 01_data_preparation:

- Данные успешно загружены и имеют корректные размеры и структуру.
- Проведена базовая очистка данных, включая замену пропущенных значений и преобразование типов данных.
- Оптимизированы типы данных, что позволило существенно сократить объем используемой памяти и повысить производительность работы с датасетами.
- Подготовленные и оптимизированные датасеты сохранены в формате parquet и готовы к дальнейшему разведочному анализу (EDA) и генерации признаков.

На следующем этапе (ноутбук 02_eda_and_feature_analysis.ipynb) мы выполним глубокий разведочный анализ и сгенерируем дополнительные признаки, избегая утечек данных.